
# Upside Thesis / Downside Thesis Automation Notebook

This notebook automates the generation of **Upside Thesis / Downside Thesis** summaries for US companies using:

- An Excel-based extraction framework which are essential attributes around which oytput needs to be generated
- Excel file has name of dimensions, associated prompts, keywords and semnatic search queries to filter relevant pages from 10-K & 10-Q
- Company 10-K and 10-Q filings (PDF)
- LLM (Semantic Search, filing-grounded)

## Two-Step Process:

**Step 1 (Per Dimension):**
- Split PDF by pages 
- Filter pages by keywords to identify relevant sections which can go to Semnatic Search
- Further filter pages by Semantic Search so ony relavant 10 pages are fed to LLM  
- Extract facts from 10-K/10-Q filings based on dimesion prompts in the excel
- Generate Upside Thesis and Downside Thesis interpretations with intensity ratings (High/Medium/Low)
- Returns JSON with facts, upside_thesis, and downside_thesis arrays

**Step 2 (Final Summary):**
- Synthesize all N dimension JSONs into a cohesive Equity Analyst summary
- Prioritize high-intensity points
- Output professional Upside Thesis / Downside Thesis narrative



## 1. Install & Import Dependencies

In [15]:

# %pip install pandas openpyxl pypdf azure-identity azure-ai-openai langchain


In [16]:
import os
import json
import pandas as pd
from pypdf import PdfReader
from typing import Dict, List
import numpy as np


## 2. Configuration (PLACEHOLDERS)

In [ ]:

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT='https://your-resource.openai.azure.com'
AZURE_OPENAI_API_KEY='your-api-key-here'
AZURE_API_VERSION='2024-02-15-preview'
AZURE_DEPLOYMENT_NAME='your-gpt4-deployment-name'
AZURE_EMBEDDING_DEPLOYMENT='your-embedding-deployment-name'

FRAMEWORK_EXCEL_PATH = "Upside_Downside_Thesis_10K_10Q_Extraction_Framework.xlsx"
FILINGS_DIR = "./SEC_Filings"
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Global embedding cache to avoid recomputation
EMBEDDING_CACHE = {}

# Global tracking metrics
METRICS = {
    'llm_calls': 0,
    'embedding_calls': 0,
    'total_tokens': 0,
    'prompt_tokens': 0,
    'completion_tokens': 0,
    'embedding_tokens': 0,
    'start_time': None,
    'end_time': None
}


## 3. Load Upside/Downside Thesis Extraction Framework

In [18]:

framework_df = pd.read_excel(FRAMEWORK_EXCEL_PATH)
framework_df.head(20)


,Dimension,Extraction Prompt,Keyword_Filter,Semantic_Query,Min_Relevant_Pages
0,Business Model & Competitive Position,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"business model, core operations, operating mod...","How the company describes its business model, ...",3
1,Revenue Growth Drivers & Demand Outlook,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"revenue drivers, demand trends, backlog, booki...",What factors management identifies as driving ...,3
2,Pricing Power & Revenue Quality,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"pricing power, price increases, pricing action...",What factors management identifies as driving ...,2
3,Margins & Cost Structure,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"gross margin, operating margin, operating inco...",What factors management cites as affecting gro...,2
4,Cash Flow Generation,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"operating cash flow, free cash flow, cash conv...",How management describes capital allocation pr...,2
5,Capital Allocation & Investment Discipline,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"capital allocation, investment priorities, acq...",How management describes capital allocation pr...,2
6,Balance Sheet Strength & Liquidity,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"balance sheet, liquidity position, leverage, d...","How management discusses liquidity, cash gener...",2
7,Operational & Supply Chain Risks,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"supply chain disruption, operational risk, cap...",Material risks management identifies that coul...,2
8,"Market, Macro & Regulatory Exposure",Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"macroeconomic conditions, interest rates, infl...",What management discloses about this dimension...,2
9,Management Strategy & Execution,Task:\n\nStep 1 – Fact Extraction: \nExtract v...,"management strategy, strategic priorities, exe...",What management discloses about this dimension...,2


## 4. PDF Parsing Utilities

In [19]:

def extract_text_from_pdf(pdf_path: str) -> List[Dict]:
    """
    Extract text from PDF at PAGE LEVEL with metadata.
    Returns list of dicts: [{'page_num': int, 'text': str, 'filing_type': str}, ...]
    """
    reader = PdfReader(pdf_path)
    pages = []
    
    # Infer filing type from filename
    filing_type = "Unknown"
    if "10-K" in pdf_path:
        filing_type = "10-K"
    elif "10-Q" in pdf_path:
        filing_type = "10-Q"
    
    for page_num, page in enumerate(reader.pages, start=1):
        text = page.extract_text()
        if text:
            pages.append({
                'page_num': page_num,
                'text': text,
                'filing_type': filing_type
            })
    
    return pages


In [20]:

def filter_pages_by_keywords(pages: List[Dict], keywords: str) -> List[Dict]:
    """
    Filter pages containing ANY keyword from the comma-separated list (case-insensitive).
    Returns filtered pages with metadata intact.
    """
    if not keywords or keywords.strip() == "":
        return pages
    
    keyword_list = [kw.strip().lower() for kw in keywords.split(',') if kw.strip()]
    
    filtered = []
    for page in pages:
        page_text_lower = page['text'].lower()
        if any(kw in page_text_lower for kw in keyword_list):
            filtered.append(page)
    
    return filtered


def embed_texts_with_azure_openai(texts: List[str], max_retries: int = 3) -> List[List[float]]:
    """
    Generate embeddings using Azure OpenAI.
    Uses EMBEDDING_CACHE to avoid recomputation.
    Implements retry logic with exponential backoff for rate limit errors.
    """
    from openai import AzureOpenAI
    import time
    
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    
    embeddings = []
    texts_to_embed = []
    cached_indices = []
    
    # Check cache
    for i, text in enumerate(texts):
        text_hash = hash(text)
        if text_hash in EMBEDDING_CACHE:
            embeddings.append(EMBEDDING_CACHE[text_hash])
            cached_indices.append(i)
        else:
            texts_to_embed.append((i, text, text_hash))
    
    # Embed uncached texts with retry logic
    if texts_to_embed:
        batch_texts = [t[1] for t in texts_to_embed]
        
        for attempt in range(max_retries):
            try:
                response = client.embeddings.create(
                    model=AZURE_EMBEDDING_DEPLOYMENT,
                    input=batch_texts
                )
                
                # Track embedding call
                METRICS['embedding_calls'] += 1
                METRICS['embedding_tokens'] += response.usage.total_tokens
                
                for idx, (original_idx, text, text_hash) in enumerate(texts_to_embed):
                    embedding = response.data[idx].embedding
                    EMBEDDING_CACHE[text_hash] = embedding
                    embeddings.insert(original_idx, embedding)
                
                return embeddings
                
            except Exception as e:
                error_str = str(e)
                
                # Check if it's a rate limit error (429)
                if "429" in error_str or "RateLimitReached" in error_str:
                    wait_time = (2 ** attempt) * 15  # 15s, 30s, 60s
                    print(f"⚠️  Rate limit hit. Waiting {wait_time}s before retry {attempt + 1}/{max_retries}...")
                    time.sleep(wait_time)
                    continue
                else:
                    print(f"❌ Error generating embeddings: {e}")
                    print(f"⚠️  Falling back to zero vectors - semantic search will be ineffective!")
                    return [[0.0] * 1536 for _ in texts]
        
        # If all retries exhausted
        print(f"❌ Rate limit retries exhausted after {max_retries} attempts.")
        print(f"⚠️  Semantic search will use arbitrary pages instead of semantically relevant ones!")
        return [[0.0] * 1536 for _ in texts]
    
    return embeddings


def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    """Compute cosine similarity between two vectors."""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return np.dot(vec1, vec2) / (norm1 * norm2)


def retrieve_top_k_pages_by_embedding(
    pages: List[Dict], 
    semantic_query: str, 
    k: int = 15
) -> List[Dict]:
    """
    Retrieve top-k pages by semantic similarity using Azure OpenAI embeddings.
    Computes cosine similarity between query and page embeddings.
    """
    if not pages:
        return []
    
    # Generate embeddings
    page_texts = [p['text'] for p in pages]
    all_texts = [semantic_query] + page_texts
    
    embeddings = embed_texts_with_azure_openai(all_texts)
    
    query_embedding = embeddings[0]
    page_embeddings = embeddings[1:]
    
    # Compute similarity scores
    scores = []
    for i, page_emb in enumerate(page_embeddings):
        similarity = cosine_similarity(query_embedding, page_emb)
        scores.append((similarity, i))
    
    # Sort by similarity (descending) and take top-k
    scores.sort(reverse=True, key=lambda x: x[0])
    top_k_indices = [idx for _, idx in scores[:k]]
    
    # Return top-k pages with similarity scores
    top_pages = []
    for score, idx in scores[:k]:
        page = pages[idx].copy()
        page['similarity_score'] = score
        top_pages.append(page)
    
    return top_pages


def should_skip_dimension(pages: List[Dict], min_pages: int) -> bool:
    """
    Check if dimension should be skipped due to insufficient evidence.
    """
    return len(pages) < min_pages


def build_dimension_context(pages: List[Dict]) -> str:
    """
    Build dimension-scoped context from retrieved pages.
    Format: [Filing Type - Page N]\n{text}\n\n
    """
    if not pages:
        return ""
    
    context_parts = []
    for page in pages:
        header = f"[{page['filing_type']} - Page {page['page_num']}]"
        context_parts.append(f"{header}\n{page['text']}")
    
    return "\n\n".join(context_parts)


## 5. Retrieval and Filtering Layer (RAG + Hallucination Prevention)

This section implements:
- Keyword-based filtering
- Semantic vector search using Azure OpenAI embeddings
- Cosine similarity ranking
- Context building from relevant pages

## 6. Load Company Filings

In [21]:

def load_company_filings(ticker: str) -> Dict[str, List[Dict]]:
    """
    Load 10-K and 10-Q filings for a given ticker at PAGE LEVEL.
    Returns: {'10-K': [page_dicts], '10-Q': [page_dicts]}
    Each page_dict: {'page_num': int, 'text': str, 'filing_type': str}
    """
    import glob
    
    filings = {}
    
    # Find 10-K files (may have dates in filename)
    ten_k_pattern = os.path.join(FILINGS_DIR, f"{ticker}_10-K*.pdf")
    ten_k_files = glob.glob(ten_k_pattern)
    if ten_k_files:
        filings['10-K'] = extract_text_from_pdf(ten_k_files[0])
    
    # Find 10-Q files (may have dates in filename)
    ten_q_pattern = os.path.join(FILINGS_DIR, f"{ticker}_10-Q*.pdf")
    ten_q_files = glob.glob(ten_q_pattern)
    if ten_q_files:
        filings['10-Q'] = extract_text_from_pdf(ten_q_files[0])
    
    return filings


## 7. Azure OpenAI Call (PLACEHOLDER)

In [22]:

def call_azure_openai(prompt: str, context: str) -> str:
    """
    Call Azure OpenAI with the given prompt and context.
    """
    from openai import AzureOpenAI
    
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    
    # Combine prompt and context into messages
    messages = [
        {"role": "system", "content": "You are a financial analyst assistant that extracts information from SEC filings."},
        {"role": "user", "content": f"{prompt}\n\nContext:\n{context}"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=messages,
            temperature=0.1,
            max_tokens=4000
        )
        
        # Track LLM call and tokens
        METRICS['llm_calls'] += 1
        METRICS['prompt_tokens'] += response.usage.prompt_tokens
        METRICS['completion_tokens'] += response.usage.completion_tokens
        METRICS['total_tokens'] += response.usage.total_tokens
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e}")
        return "{\"facts\": [], \"upside_thesis\": [], \"downside_thesis\": []}"


## 8. Step 1: Dimension Extraction with Retrieval

Extracts facts and generates Upside/Downside Thesis interpretations for each dimension using:
- Keyword filtering and semantic search
- Intensity ratings (High/Medium/Low)

In [23]:

def run_dimension_extraction(row, filings_pages: Dict[str, List[Dict]], dimension_num: int = 0, ticker: str = "") -> Dict:
    """
    Step 1: Extract facts and generate Upside/Downside Thesis interpretations with intensity.
    NOW WITH RETRIEVAL LAYER to prevent hallucinations.
    Returns JSON with facts, upside_thesis, and downside_thesis arrays.
    """
    dimension_name = row['Dimension']
    prompt = row['Extraction Prompt']
    
    # Read retrieval configuration from framework
    keyword_filter = row.get('Keyword_Filter', '')
    semantic_query = row.get('Semantic_Query', '')
    min_relevant_pages = int(row.get('Min_Relevant_Pages', 3))
    
    print(f"\n{'='*80}")
    print(f"Running Dimension {dimension_num}: {dimension_name} for {ticker} company")
    print(f"{'='*80}")
    print(f"Retrieval Config:")
    print(f"  - Keyword Filter: {keyword_filter[:100]}...")
    print(f"  - Semantic Query: {semantic_query[:100]}...")
    print(f"  - Min Relevant Pages: {min_relevant_pages}")
    
    # Combine all pages from all filings
    all_pages = []
    for filing_type, pages in filings_pages.items():
        all_pages.extend(pages)
    
    print(f"\nTotal pages available: {len(all_pages)}")
    
    # STEP 1: Keyword filtering
    print(f"Applying keyword filter...")
    filtered_pages = filter_pages_by_keywords(all_pages, keyword_filter)
    print(f"Pages after keyword filter: {len(filtered_pages)}")
    
    if not filtered_pages:
        print(f"⚠️ No pages matched keywords. Skipping dimension.")
        return {
            "dimension": dimension_name,
            "facts": [],
            "upside_thesis": [{"interpretation": "Insufficient disclosure for this dimension.", "intensity": "N/A"}],
            "downside_thesis": [{"interpretation": "Insufficient disclosure for this dimension.", "intensity": "N/A"}]
        }
    
    # STEP 2: Semantic vector search
    print(f"Performing semantic vector search...")
    top_pages = retrieve_top_k_pages_by_embedding(filtered_pages, semantic_query, k=10)
    print(f"Top-k pages selected: {len(top_pages)}")
    
    if top_pages:
        scores_str = [f"{p['similarity_score']:.3f}" for p in top_pages[:3]]
        print(f"Similarity scores: {scores_str}")
    
    # STEP 3: Hallucination guardrail
    if should_skip_dimension(top_pages, min_relevant_pages):
        print(f"⚠️ Insufficient pages ({len(top_pages)} < {min_relevant_pages}). Skipping LLM call.")
        return {
            "dimension": dimension_name,
            "facts": [],
            "upside_thesis": [{"interpretation": "Insufficient disclosure for this dimension.", "intensity": "N/A"}],
            "downside_thesis": [{"interpretation": "Insufficient disclosure for this dimension.", "intensity": "N/A"}]
        }
    
    # STEP 4: Build dimension-scoped context
    context = build_dimension_context(top_pages)
    
    print(f"\nContext built (length: {len(context)} characters)")
    pages_info = [f"{p['filing_type']}-P{p['page_num']}" for p in top_pages]
    print(f"Pages included: {pages_info}")
    
    # STEP 5: Modify prompt with hallucination prevention instructions
    enhanced_prompt = prompt + "\n\n" + "Use ONLY the provided excerpts. Do NOT infer beyond them."
    
    print(f"\nCalling Azure OpenAI with enhanced prompt...")
    
    response = call_azure_openai(enhanced_prompt, context)
    
    print(f"\nResponse received (length: {len(response)} chars)")
    print(f"Response preview:\n{response[:300]}...")
    
    # Parse JSON response from LLM
    try:
        parsed = json.loads(response)
        
        # Handle case where LLM returns nested JSON string
        if isinstance(parsed, str):
            print("⚠️ Response was a JSON string, attempting to parse again...")
            parsed = json.loads(parsed)
        
        # Verify it's a dictionary
        if not isinstance(parsed, dict):
            raise ValueError(f"Expected dict, got {type(parsed)}")
        
        num_facts = len(parsed.get('facts', []))
        num_upside = len(parsed.get('upside_thesis', []))
        num_downside = len(parsed.get('downside_thesis', []))
        print(f"\n✓ Successfully parsed JSON:")
        print(f"  - Facts extracted: {num_facts}")
        print(f"  - Upside Thesis points: {num_upside}")
        print(f"  - Downside Thesis points: {num_downside}")
        return parsed
    except (json.JSONDecodeError, ValueError) as e:
        print(f"\n✗ JSON parsing failed: {e}")
        print(f"Response type: {type(response)}")
        print(f"Response content: {response[:500]}")
        print(f"Returning empty result for dimension: {dimension_name}")
        # Fallback if JSON parsing fails
        return {
            "facts": [],
            "upside_thesis": [],
            "downside_thesis": []
        }


## 9. Step 2: Final Summary Generation

Synthesizes all dimension JSONs into a cohesive Equity Analyst style analyst summary with separate Upside Thesis and Downside Thesis sections.

In [ ]:

def generate_final_summary(ticker: str, all_dimensions_data: List[Dict]) -> Dict[str, str]:
    """
    Step 2: Generate final  Equity style analyst summary from all dimension extractions.
    Takes the JSON output from Step 1 for all N dimensions and synthesizes it.
    """
    
    # Prepare the consolidated JSON from all dimensions
    dimensions_summary = json.dumps(all_dimensions_data, indent=2)
    
    synthesis_prompt = f"""
You are a Equity Analyst writing an Upside Thesis / Downside Thesis summary for {ticker}.

Below is the complete JSON output from filing-grounded analysis across all investment dimensions:

{dimensions_summary}

Task:
Write a concise, professional Upside Thesis / Downside Thesis summary as it would appear in equity research report. 

UPSIDE THESIS:
- Format as BULLET POINTS using markdown (• or -)
- Order points by DECREASING IMPORTANCE: High intensity first, then Medium, then Low
- Synthesize the optimistic interpretations across all dimensions
- Write in the voice of an Equity analyst
- Use only the interpretations provided in the JSON
- Keep it concise (3-5 key points)
- Each bullet point should be a complete, clear statement

DOWNSIDE THESIS:
- Format as BULLET POINTS using markdown (• or -)
- Order points by DECREASING IMPORTANCE: High intensity first, then Medium, then Low
- Synthesize the cautious interpretations across all dimensions
- Write in the voice of an Equity analyst
- Use only the interpretations provided in the JSON
- Keep it concise (3-5 key points)
- Each bullet point should be a complete, clear statement

Rules:
- MUST use bullet point format (start each point with • or -)
- Do NOT add facts not present in the JSON
- Do NOT make forecasts or valuations
- Maintain professional, analytical tone
- If a dimension has "Not disclosed", acknowledge gaps where material
- STRICTLY order by intensity: High → Medium → Low

Output Format:
UPSIDE THESIS:
• [High intensity point 1]
• [High intensity point 2]
• [Medium intensity point]
...

DOWNSIDE THESIS:
• [High intensity point 1]
• [High intensity point 2]
• [Medium intensity point]
...
"""
    
    output = call_azure_openai(synthesis_prompt, dimensions_summary)
    
    # Parse the output to separate Upside Thesis and Downside Thesis sections
    upside_summary = ""
    downside_summary = ""
    
    # Try to split by common section headers
    if "UPSIDE THESIS" in output.upper() and "DOWNSIDE THESIS" in output.upper():
        # Find the positions of the headers (case-insensitive)
        import re
        upside_match = re.search(r'UPSIDE THESIS:?', output, re.IGNORECASE)
        downside_match = re.search(r'DOWNSIDE THESIS:?', output, re.IGNORECASE)
        
        if upside_match and downside_match:
            # Extract text between UPSIDE THESIS and DOWNSIDE THESIS
            upside_start = upside_match.end()
            downside_start = downside_match.start()
            upside_summary = output[upside_start:downside_start].strip()
            
            # Extract text after DOWNSIDE THESIS
            downside_summary = output[downside_match.end():].strip()
        else:
            # Fallback: use entire output for both
            upside_summary = output
            downside_summary = output
    else:
        # Fallback: use entire output for both
        upside_summary = output
        downside_summary = output
    
    # Parse the output (expecting Upside Thesis and Downside Thesis sections)
    return {
        "upside_thesis_summary": upside_summary,
        "downside_thesis_summary": downside_summary,
        "raw_dimensions": all_dimensions_data
    }


## 10. End-to-End Company Processing

Main orchestration function that runs both Step 1 (dimension extraction) and Step 2 (final summary) for a given company ticker.

In [ ]:

def process_company(ticker: str):
    """
    End-to-end processing for a single company.
    
    Step 1: Run dimension extraction for all N dimensions (returns JSON per dimension)
    Step 2: Generate final Equity-style Upside Thesis / Downside Thesis summary
    """
    import time
    
    # Reset and start tracking
    METRICS['llm_calls'] = 0
    METRICS['embedding_calls'] = 0
    METRICS['total_tokens'] = 0
    METRICS['prompt_tokens'] = 0
    METRICS['completion_tokens'] = 0
    METRICS['embedding_tokens'] = 0
    METRICS['start_time'] = time.time()
    
    # Load company filings
    filings = load_company_filings(ticker)
    
    # Step 1: Run extraction for all dimensions
    all_dimensions_data = []
    for dim_idx, (_, row) in enumerate(framework_df.iterrows(), start=1):
        dimension_name = row['Dimension']
        extracted = run_dimension_extraction(row, filings, dimension_num=dim_idx, ticker=ticker)
        
        # Add dimension name to the extracted data
        extracted['dimension'] = dimension_name
        all_dimensions_data.append(extracted)
    
    # Step 2: Generate final summary from all dimensions
    final_summary = generate_final_summary(ticker, all_dimensions_data)
    
    # End tracking
    METRICS['end_time'] = time.time()
    
    return final_summary


## 11. Save Results

Saves analysis results as JSON and Excel files with:
- Final Summary sheet (Upside/Downside Thesis)
- Dimension Details sheet (per-dimension facts and interpretations)

In [26]:

def save_results(ticker: str, results: Dict):
    """Save results as both JSON and Excel files"""
    # Save JSON
    json_file = os.path.join(OUTPUT_DIR, f"{ticker}_upside_downside_thesis.json")
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # Save Excel with multiple sheets
    excel_file = os.path.join(OUTPUT_DIR, f"{ticker}_upside_downside_thesis.xlsx")
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Sheet 1: Final Summary
        # Handle both single string and dict with numeric keys (old format)
        upside_summary = results['upside_thesis_summary']
        downside_summary = results['downside_thesis_summary']
        
        # If old format (dict with numeric keys), get the first entry
        if isinstance(upside_summary, dict):
            upside_summary = upside_summary.get('0', upside_summary.get(0, str(upside_summary)))
        if isinstance(downside_summary, dict):
            downside_summary = downside_summary.get('0', downside_summary.get(0, str(downside_summary)))
        
        summary_data = {
            'Section': ['Upside Thesis', 'Downside Thesis'],
            'Summary': [upside_summary, downside_summary]
        }
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_excel(writer, sheet_name='Final Summary', index=False)
        
        # Sheet 2: Dimension Details
        dimension_rows = []
        
        # Handle both list format (new) and dict format with numeric keys (old)
        raw_dims = results['raw_dimensions']
        if isinstance(raw_dims, dict):
            # Old format: convert dict with numeric keys to list
            dimensions_list = [raw_dims[str(i)] for i in range(len(raw_dims))]
        else:
            # New format: already a list
            dimensions_list = raw_dims
        
        for dim in dimensions_list:
            # Extract facts
            facts_text = "\n\n".join([f"• {f['statement']} (Source: {f['source_section']})" 
                                      for f in dim.get('facts', [])])
            
            # Extract upside thesis
            upside_text = "\n\n".join([f"[{b['intensity']}] {b['interpretation']}" 
                                      for b in dim.get('upside_thesis', [])])
            
            # Extract downside thesis
            downside_text = "\n\n".join([f"[{b['intensity']}] {b['interpretation']}" 
                                      for b in dim.get('downside_thesis', [])])
            
            dimension_rows.append({
                'Dimension': dim.get('dimension', ''),
                'Facts Extracted': facts_text if facts_text else 'No facts extracted',
                'Upside Thesis': upside_text if upside_text else 'No upside points',
                'Downside Thesis': downside_text if downside_text else 'No downside points'
            })
        
        df_dimensions = pd.DataFrame(dimension_rows)
        df_dimensions.to_excel(writer, sheet_name='Dimension Details', index=False)
        
        # Adjust column widths
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            for column in worksheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = min(max_length + 2, 100)
                worksheet.column_dimensions[column_letter].width = adjusted_width
    
    print(f"\n{'='*80}")
    print(f"✓ Saved JSON to: {json_file}")
    print(f"✓ Saved Excel to: {excel_file}")
    print(f"{'='*80}")


In [27]:

def print_usage_metrics(ticker: str = None, save_to_file: bool = True):
    """
    Display comprehensive usage metrics and cost estimates.
    Optionally save to output files.
    
    Pricing (as of Dec 2025):
    - GPT-4.1 Mini: $0.30/1M input tokens, $1.20/1M output tokens
    - text-embedding-3-small: $0.02/1M tokens
    
    Args:
        ticker: Company ticker symbol (for filename)
        save_to_file: If True, saves metrics to JSON and TXT files
    """
    if METRICS['start_time'] is None:
        print("No metrics available. Run process_company() first.")
        return
    
    # Calculate duration
    duration = METRICS['end_time'] - METRICS['start_time']
    minutes = int(duration // 60)
    seconds = int(duration % 60)
    
    # Calculate costs (Azure OpenAI pricing)
    llm_input_cost = (METRICS['prompt_tokens'] / 1_000_000) * 0.30
    llm_output_cost = (METRICS['completion_tokens'] / 1_000_000) * 1.20
    embedding_cost = (METRICS['embedding_tokens'] / 1_000_000) * 0.02
    total_cost = llm_input_cost + llm_output_cost + embedding_cost
    
    # Build metrics dictionary
    metrics_data = {
        'ticker': ticker,
        'timestamp': pd.Timestamp.now().isoformat(),
        'api_calls': {
            'llm_calls': METRICS['llm_calls'],
            'embedding_calls': METRICS['embedding_calls'],
            'total_api_calls': METRICS['llm_calls'] + METRICS['embedding_calls']
        },
        'token_usage': {
            'prompt_tokens': METRICS['prompt_tokens'],
            'completion_tokens': METRICS['completion_tokens'],
            'embedding_tokens': METRICS['embedding_tokens'],
            'total_llm_tokens': METRICS['total_tokens'],
            'grand_total_tokens': METRICS['total_tokens'] + METRICS['embedding_tokens']
        },
        'execution_time': {
            'duration_seconds': round(duration, 1),
            'duration_formatted': f"{minutes}m {seconds}s",
            'avg_per_llm_call_seconds': round(duration / METRICS['llm_calls'], 1)
        },
        'costs_usd': {
            'llm_input': round(llm_input_cost, 4),
            'llm_output': round(llm_output_cost, 4),
            'embeddings': round(embedding_cost, 4),
            'total_cost': round(total_cost, 4)
        },
        'efficiency': {
            'embedding_cache_hits': len(EMBEDDING_CACHE),
            'tokens_per_llm_call': round(METRICS['total_tokens'] / METRICS['llm_calls'], 0),
            'cost_per_llm_call': round((llm_input_cost + llm_output_cost) / METRICS['llm_calls'], 4)
        }
    }
    
    # Print to console
    print(f"\n{'='*80}")
    print(f"USAGE METRICS & COST SUMMARY")
    print(f"{'='*80}")
    print(f"\n📊 API CALLS:")
    print(f"  • LLM Calls (Chat):        {METRICS['llm_calls']}")
    print(f"  • Embedding Calls:         {METRICS['embedding_calls']}")
    print(f"  • Total API Calls:         {METRICS['llm_calls'] + METRICS['embedding_calls']}")
    
    print(f"\n🔢 TOKEN USAGE:")
    print(f"  • Prompt Tokens:           {METRICS['prompt_tokens']:,}")
    print(f"  • Completion Tokens:       {METRICS['completion_tokens']:,}")
    print(f"  • Embedding Tokens:        {METRICS['embedding_tokens']:,}")
    print(f"  • Total LLM Tokens:        {METRICS['total_tokens']:,}")
    print(f"  • Grand Total Tokens:      {METRICS['total_tokens'] + METRICS['embedding_tokens']:,}")
    
    print(f"\n⏱️  EXECUTION TIME:")
    print(f"  • Total Duration:          {minutes}m {seconds}s ({duration:.1f}s)")
    print(f"  • Avg per LLM Call:        {duration / METRICS['llm_calls']:.1f}s")
    
    print(f"\n💰 ESTIMATED COST (Azure OpenAI):")
    print(f"  • LLM Input:               ${llm_input_cost:.4f}")
    print(f"  • LLM Output:              ${llm_output_cost:.4f}")
    print(f"  • Embeddings:              ${embedding_cost:.4f}")
    print(f"  • TOTAL COST:              ${total_cost:.4f}")
    
    print(f"\n📈 EFFICIENCY METRICS:")
    print(f"  • Embedding Cache Hits:    {len(EMBEDDING_CACHE)} unique texts cached")
    print(f"  • Tokens per LLM Call:     {METRICS['total_tokens'] / METRICS['llm_calls']:.0f} avg")
    print(f"  • Cost per LLM Call:       ${(llm_input_cost + llm_output_cost) / METRICS['llm_calls']:.4f} avg")
    
    print(f"{'='*80}\n")
    
    # Save to files if requested
    if save_to_file:
        # Generate filenames
        ticker_prefix = f"{ticker}_" if ticker else ""
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        
        # Save readable text file only (no JSON)
        txt_filename = os.path.join(OUTPUT_DIR, f"{ticker_prefix}usage_metrics_{timestamp}.txt")
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"USAGE METRICS & COST SUMMARY - {ticker or 'Unknown Ticker'}\n")
            f.write(f"Generated: {metrics_data['timestamp']}\n")
            f.write("="*80 + "\n\n")
            
            f.write("API CALLS:\n")
            f.write(f"  • LLM Calls (Chat):        {METRICS['llm_calls']}\n")
            f.write(f"  • Embedding Calls:         {METRICS['embedding_calls']}\n")
            f.write(f"  • Total API Calls:         {METRICS['llm_calls'] + METRICS['embedding_calls']}\n\n")
            
            f.write("TOKEN USAGE:\n")
            f.write(f"  • Prompt Tokens:           {METRICS['prompt_tokens']:,}\n")
            f.write(f"  • Completion Tokens:       {METRICS['completion_tokens']:,}\n")
            f.write(f"  • Embedding Tokens:        {METRICS['embedding_tokens']:,}\n")
            f.write(f"  • Total LLM Tokens:        {METRICS['total_tokens']:,}\n")
            f.write(f"  • Grand Total Tokens:      {METRICS['total_tokens'] + METRICS['embedding_tokens']:,}\n\n")
            
            f.write("EXECUTION TIME:\n")
            f.write(f"  • Total Duration:          {minutes}m {seconds}s ({duration:.1f}s)\n")
            f.write(f"  • Avg per LLM Call:        {duration / METRICS['llm_calls']:.1f}s\n\n")
            
            f.write("ESTIMATED COST (Azure OpenAI):\n")
            f.write(f"  • LLM Input:               ${llm_input_cost:.4f}\n")
            f.write(f"  • LLM Output:              ${llm_output_cost:.4f}\n")
            f.write(f"  • Embeddings:              ${embedding_cost:.4f}\n")
            f.write(f"  • TOTAL COST:              ${total_cost:.4f}\n\n")
            
            f.write("EFFICIENCY METRICS:\n")
            f.write(f"  • Embedding Cache Hits:    {len(EMBEDDING_CACHE)} unique texts cached\n")
            f.write(f"  • Tokens per LLM Call:     {METRICS['total_tokens'] / METRICS['llm_calls']:.0f} avg\n")
            f.write(f"  • Cost per LLM Call:       ${(llm_input_cost + llm_output_cost) / METRICS['llm_calls']:.4f} avg\n")
            f.write("="*80 + "\n")
        
        print(f"💾 Metrics saved to: {txt_filename}\n")
    
    return metrics_data


## 13. Run Analysis for Companies

Execute the complete analysis pipeline for multiple company tickers and display results.

## 12. Usage Metrics and Cost Tracking

Tracks and reports:
- API call counts (LLM + embeddings)
- Token usage and costs
- Execution time and efficiency metrics

In [28]:
ticker_list=['BLDR','CEG','CSW','DOCN','FTAI','LSCC','TRGP']
#ticker_list=['BLDR']
for ticker in ticker_list:
    results = process_company(ticker)
    save_results(ticker, results)
    # Display usage metrics and save to files
    metrics = print_usage_metrics(ticker=ticker, save_to_file=True)
    # Display the final summary
    print("\n" + "="*80)
    print("FINAL UPSIDE THESIS / DOWNSIDE THESIS SUMMARY")
    print("="*80)
    print("\n📈 UPSIDE THESIS:")
    print("-" * 80)
    print(results['upside_thesis_summary'])
    print("\n📉 DOWNSIDE THESIS:")
    print("-" * 80)
    print(results['downside_thesis_summary'])


Running Dimension 1: Business Model & Competitive Position for BLDR company
Retrieval Config:
  - Keyword Filter: business model, core operations, operating model, revenue model, asset portfolio, operating assets, ...
  - Semantic Query: How the company describes its business model, competitive advantages, scale, customer value proposit...
  - Min Relevant Pages: 3

Total pages available: 169
Applying keyword filter...
Pages after keyword filter: 11
Performing semantic vector search...
Top-k pages selected: 10
Similarity scores: ['0.469', '0.460', '0.411']

Context built (length: 51780 characters)
Pages included: ['10-K-P12', '10-K-P10', '10-K-P22', '10-K-P18', '10-K-P44', '10-K-P26', '10-K-P20', '10-Q-P29', '10-Q-P31', '10-K-P56']

Calling Azure OpenAI with enhanced prompt...

Response received (length: 11725 chars)
Response preview:
{
  "facts": [
    {
      "statement": "Our strategy is to attract and retain customers through exceptional customer service, leading product quality, 